In [23]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
location = 'data/'
auctionsFeaturesFile = location + 'auctionsFeaturesFirstThreeDays.csv'
eventsFeaturesFile = location + 'eventsFeaturesFirstThreeDays.csv'
clicksFeaturesFile = location + 'clicksFeaturesFirstThreeDays.csv'
installsFeaturesFile = location + 'installsFeaturesFirstThreeDays.csv'
targetFile= location + 'target_competencia_ids.csv'

In [3]:
auctionsFeatures=pd.read_csv(auctionsFeaturesFile)
eventsFeatures=pd.read_csv(eventsFeaturesFile)
clicksFeatures=pd.read_csv(clicksFeaturesFile)
installsFeatures=pd.read_csv(installsFeaturesFile)

In [78]:
target = pd.read_csv(targetFile)
target['ref_hash']=target['ref_hash'].str.strip('_sc')
target['ref_hash']=target['ref_hash'].str.strip('_st')
target=pd.DataFrame(target['ref_hash'].unique())
target['value']=0
target.columns=['device_id','value']
target=target.astype({'device_id':'int64','value':'float'})
target.head()

,device_id,value
0,1000169251625791246,0.0
1,1000395625957344683,0.0
2,1003027494996471685,0.0
3,1006670001679961544,0.0
4,1007573308966476713,0.0


In [6]:
labelsFile=location + 'labelsFirstThreeDays.csv'
labels=pd.read_csv(labelsFile)

In [7]:
labelsSC=labels.loc[labels['sc'].notnull()]
labelsSC=labelsSC.drop('st',axis=1)
labelsST=labels.loc[labels['st'].notnull()]
labelsST=labelsST.drop('sc',axis=1)

In [9]:
labels=labels.fillna(259200)
labels.head()

,device_id,st,sc
0,5857744372586891366,259200.000000,259200.0
1,7642521036780133571,436.543534,259200.0
2,2548841562898283198,259200.000000,259200.0
3,609402887625919085,259200.000000,259200.0
4,9114651763556439823,259200.000000,259200.0


In [8]:
labelsSC.count()#56850
labelsST.count()#251508

device_id    251508
st           251508
dtype: int64

In [13]:
features=auctionsFeatures.merge(eventsFeatures,how='outer')
features=features.merge(clicksFeatures,how='outer')
features=features.merge(installsFeatures,how='outer')

In [14]:
features.head()

,device_id,auctionsMeanHour,auctionsStdDevHour,auctionsCount,auctionsMostFreqDay,auctionsMeanInstant,auctionsStdDevInstant,auctionsTopSecond,auctionsMeanInterval,auctionsAproxFreq,...,clicksMeanInterval,clicksAproxFreq,clicksStdDevInterval,clicksLast,installsCount,installsMostFreqDay,installsMeanInterval,installsAproxFreq,installsStdDevInterval,installsLast
0,1109595589636746168,20.344831,5.775980,128.0,3.0,217691.904097,78348.145610,252711.397215,2024.923399,0.000494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5896614299191635403,22.110936,5.441725,46.0,3.0,252399.878976,19590.204861,257059.673527,5631.657083,0.000178,...,NaN,NaN,NaN,NaN,1.0,19.0,56.0,0.017857,NaN,56.0
2,4172466725848941608,21.253534,5.540618,252.0,3.0,227027.493094,65290.937857,253271.814659,1028.449490,0.000972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2616279795187318849,16.785580,7.204283,191.0,1.0,128734.373984,72254.139672,155144.580698,1356.284096,0.000737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8034952072073026056,9.002714,6.712023,166.0,2.0,117769.315541,59901.949880,98475.225948,1560.535351,0.000641,...,NaN,NaN,NaN,NaN,1.0,19.0,32.0,0.031250,NaN,32.0


In [183]:
#pd.Series(features['device_id'].unique())#426755
y=labelsST
X=features
X=y.merge(X,how='inner').drop('st',axis=1)
y=labelsST['st']
X=X.fillna(0)
X.loc[X['installsAproxFreq'] == float('inf'), 'installsAproxFreq'] = 0


In [188]:
X=X.transform(lambda x: (x-x.mean())/x.std())

In [241]:
#X features, y labels
#predict ST
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.05, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)   




(238932, 30) (238932,)
(12576, 30) (12576,)


In [209]:
from sklearn.linear_model import Ridge
RidgeModel=Ridge(alpha=0.1)
RidgeModel.fit(X_train,y_train)
ridge_predict=RidgeModel.predict(X_test)

In [211]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, ridge_predict)

4044658602.0892677

In [233]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RidgeModel, X,y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.13 (+/- 0.12)


In [242]:
from sklearn import linear_model
lasso = linear_model.Lasso(alpha=0.1)
lasso.fit(X_train,y_train)  
lasso_predict=lasso.predict(X_test)

/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [243]:
mean_squared_error(y_test, lasso_predict)

4044650532.046977

In [239]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lasso, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. 

Accuracy: 0.13 (+/- 0.13)


/home/nicolas/.local/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
